In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
from plotly.offline import init_notebook_mode
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

In [2]:
training_df = pd.read_csv('~/train.csv', error_bad_lines=False)

In [3]:
test_df = pd.read_csv('~/test.csv')

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning:

Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.



In [98]:
training_df['question1'][0]

'What is the step by step guide to invest in share market in india?'

In [122]:
training_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 6 columns):
id              404290 non-null int64
qid1            404290 non-null int64
qid2            404290 non-null int64
question1       404289 non-null object
question2       404288 non-null object
is_duplicate    404290 non-null int64
dtypes: int64(4), object(2)
memory usage: 18.5+ MB


In [4]:
final_corpus_question1= pd.DataFrame(columns=['question1','question2'])

In [5]:
final_corpus_question1['question1'] = training_df['question1'].append(test_df['question1'])

In [6]:
final_corpus_question1['question1'].to_csv('question1.csv')

In [7]:
final_corpus_question1['question2'] = training_df['question2'].append(test_df['question2'])

In [8]:
final_corpus_question1['question2'].to_csv('question2.csv')

In [23]:
final_corpus_question1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3967765 entries, 0 to 3563474
Data columns (total 2 columns):
question1    object
question2    object
dtypes: object(2)
memory usage: 250.8+ MB


In [9]:
import smart_open
import gensim

In [10]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [11]:
train_corpus_question1 = list(read_corpus('~/question1.csv'))

In [12]:
train_corpus_question2 = list(read_corpus('~/question2.csv'))

In [13]:
train_corpus_question1[:2]

[TaggedDocument(words=['what', 'is', 'the', 'step', 'by', 'step', 'guide', 'to', 'invest', 'in', 'share', 'market', 'in', 'india'], tags=[0]),
 TaggedDocument(words=['what', 'is', 'the', 'story', 'of', 'kohinoor', 'koh', 'noor', 'diamond'], tags=[1])]

In [14]:
final_corpus_question1_2=train_corpus_question1+train_corpus_question2

In [18]:
len(final_corpus_question1_2)

7935555

In [15]:
test=final_corpus_question1_2[:11]

In [20]:
y =training_df['is_duplicate']

In [7]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: is_duplicate, dtype: int64

In [21]:
training_df =training_df.drop(columns=['is_duplicate'])

In [27]:
from nltk.corpus import stopwords
from nltk.stem.porter import *
import nltk
nltk.download("stopwords")
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [28]:
sw_dict={}
    
for i in stopwords.words('english'):
    sw_dict[i]=1
    



In [29]:
import re
from nltk.stem import WordNetLemmatizer
#from nltk.stem import PorterStemmer
from stemming.porter2 import stem

lemi_words=[]
stem_words=[]
lmtzr = nltk.WordNetLemmatizer().lemmatize
ps = PorterStemmer()



def do_lemitise(word):
    old_word=word
    word = lmtzr(word)
    if word != old_word:
        lemi_words.append((old_word, word))
        
    return word
    
def do_stem(word):
    old_word=word
    word = stem(word)
    if word != old_word:
        stem_words.append((old_word, word))
    return word
        

def do_process_word(word):
    word = do_stem(word)
    word = do_lemitise(word)
    return word
    
    
    

def corpus_filteration(dfList):
    
    actual_list =[]
    

    
    item_num=0

    for k, v in dfList.iteritems():
        questions_corpus = re.sub('[^a-zA-Z]',' ', str(v)).lower().split()
        questions_corpus = [do_process_word(word) for word in questions_corpus if word not in sw_dict.keys()]
        
        actual_list.append(questions_corpus)
        item_num = item_num + 1
        if (item_num%1000000 == 0):
            print ("Done so far {} ".format(item_num) )
    
    return actual_list           
    
    

In [30]:
training_error1_cf = corpus_filteration(training_df['question1'])

In [ ]:
question1_check = training_error1_cf[:10]

In [31]:
training_error2_cf = corpus_filteration(training_df['question2'])

In [33]:
question2_check = training_error2_cf[:10]

In [33]:
test_question_list[0]

['step', 'step', 'guid', 'invest', 'share', 'market', 'india']

In [16]:
import gensim


In [78]:
#model = Doc2Vec([document], size = 100, window = 1, min_count = 1, workers=1)

In [24]:
from gensim.models import doc2vec
model1 = gensim.models.doc2vec.Doc2Vec(size=50, window = 1,min_count=1, epochs=4)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:570: UserWarning:

The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.



In [25]:
model1.build_vocab(final_corpus_question1_2)

In [26]:
model1.train(final_corpus_question1_2, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
'''
search_phrase = ['i', 'am', 'a', 'cat']

s1 = model.infer_vector(search_phrase, alpha=0.025, min_alpha=0.025, steps=20)
s2 = model.infer_vector(['i', 'am', 'a', 'cat'], alpha=0.025, min_alpha=0.025, steps=20)

print(cosine_similarity(s1, s2))  # Print out =  ~0.9999882
'''

In [120]:
from sklearn.metrics.pairwise import cosine_similarity
def new_distance_calculation(training_error1_cf,training_error2_cf):
    i=j=0
    s1=s2=0
    item=1
    
    #np.array(temp).reshape((len(temp), 1))
    while i <len(training_error1_cf) and j<len(training_error2_cf):
    
        search_phrase1 = training_error1_cf[i]
        search_phrase2=  training_error2_cf[j]
        #print('s_p1',search_phrase1)
        
        s1=  model1.infer_vector(search_phrase1)
        s2 = model1.infer_vector(search_phrase2)
        print('item',item)
        
        print('distance',cosine_similarity(s1.reshape(1,-1),s2.reshape(1,-1)))
        item =item+1
        search_phrase1=search_phrase2=''
        s1=s2=0
        i=i+1
        j=j+1
        
       
        
        

In [125]:
new_distance_calculation(training_error1_cf[:15],training_error2_cf[:15])


item 1
distance [[0.75892633]]
item 2
distance [[0.9348324]]
item 3
distance [[0.60062355]]
item 4
distance [[-0.09260418]]
item 5
distance [[0.17874129]]
item 6
distance [[0.8906737]]
item 7
distance [[0.7475024]]
item 8
distance [[0.67897457]]
item 9
distance [[0.9102985]]
item 10
distance [[0.8771665]]
item 11
distance [[0.7376131]]
item 12
distance [[0.3682499]]
item 13
distance [[0.7599055]]
item 14
distance [[0.79779094]]
item 15
distance [[0.9203851]]


In [127]:
training_df['is_duplicate'][:15]

0     0
1     0
2     0
3     0
4     0
5     1
6     0
7     1
8     0
9     0
10    0
11    1
12    1
13    1
14    0
Name: is_duplicate, dtype: int64

In [26]:
'''
from pyemd import emd
def distance_calculation(d1,d2):
        i=j=0
        distance_list=[]
        len_train_01 =len(d1)
        len_train_02 =len(d2)
        item_num=0
        
        
        while (i<len_train_01 and j<len_train_02):
            distance_questions =model.wv.wmdistance(d1[i], d2[j])
            #f.write(str(distance_questions))
            distance_list.append(distance_questions)
            i=i+1
            j=j+1
            item_num = item_num + 1
            if (item_num%100000 == 0):
                print ("Done so far {} ".format(item_num) )
        training_df['is_duplicate_01']=distance_list
        return training_df['is_duplicate_01']
        #return distance_list
'''

In [27]:
distance_calculation(training_error1_cf,training_error2_cf)

Done so far 100000 
Done so far 200000 
Done so far 300000 
Done so far 400000 


0         0.007921
1         0.031860
2         0.028146
3         0.059286
4         0.039029
5         0.024682
6         0.055523
7         0.028528
8         0.000000
9         0.022826
10        0.051774
11        0.036843
12        0.000000
13        0.014861
14        0.005979
15        0.040315
16        0.000000
17        0.021754
18        0.024632
19        0.011145
20        0.027057
21        0.023291
22        0.000000
23        0.056991
24        0.035852
25        0.000000
26        0.027821
27        0.043921
28        0.011563
29        0.030387
            ...   
404260    0.019989
404261    0.014949
404262    0.019387
404263    0.033201
404264    0.033290
404265    0.019032
404266    0.048689
404267    0.018818
404268    0.041726
404269    0.040590
404270    0.000000
404271    0.036030
404272    0.045595
404273    0.024746
404274    0.029124
404275    0.038635
404276    0.030007
404277         inf
404278    0.036416
404279    0.034998
404280    0.009773
404281    0.

In [32]:

item_num =0
for i, row in training_df['is_duplicate_01'].iteritems():
    if row == 0.000000 or (row >=0.012 and row <= 0.032) :
        training_df.set_value(i,'is_duplicate_01',int(1))
    else:
        training_df.set_value(i,'is_duplicate_01',int(0))
    
    item_num = item_num + 1
    if (item_num%100000 == 0):
        print ("Done so far {} ".format(item_num) )
        
   


/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:7: FutureWarning:

set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning:

set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead



Done so far 100000 
Done so far 200000 
Done so far 300000 
Done so far 400000 


In [ ]:
#accuracy_score =67.30 when dimension=100
#chaging to 300 now

In [23]:
def avg_length_corpus(training_error1_cf):
    
    min_length=0
    max_length=len(training_error1_cf[0])
    
    for i in range(0,len(training_error1_cf)):
        if len(training_error1_cf[i]) <=min_length :
            min_length= len(training_error1_cf[i])
        
    
    for i in range(1,len(training_error1_cf)):
        if len(training_error1_cf[i]) > max_length:
            max_length= len(training_error1_cf[i])
        
    
    avg_length= (min_length + max_length)//2
    
    
    return min_length,max_length,avg_length



In [20]:
avg_question1_training_set= avg_length_corpus(training_error1_cf)

In [21]:
avg_question2_training_set= avg_length_corpus(training_error2_cf)

In [27]:
def len_trainging_questions_array(training_error1_cf):
    questions_length_array =[]
    for i in training_error1_cf:
        questions_length_array.append(len(i))
        
    return questions_length_array
    

In [28]:
#question_1_length =len_trainging_questions_array(training_error1_cf)

In [29]:
question_2_length =len_trainging_questions_array(training_error2_cf)

In [30]:

import plotly.plotly as py
import plotly.graph_objs as go


trace = go.Box(
    y= question_2_length,
    name='Mean & SD',
    marker=dict(
        color='rgb(10, 140, 208)',
    ),
    boxmean='sd'
)
data = [trace]
#py.iplot(data)
plotly.offline.plot(data)


'file:///home/ubuntu/temp-plot.html'

In [51]:
def duplicate_entries(input_array):
    
    final_result=[]
    new_array=[]
    unseen=set()
    count=0
    for i in input_array:
        for j in i:
                 if j not in unseen:
                    unseen.add(j)
                    final_result.append(model[j])
                    #final_result.append(j)
        if len(final_result) >7:
            final_result=final_result[0:7]
        else:
            array_length=len(final_result)
            while array_length <7:
                final_result.append(np.zeros(50))
                array_length=array_length +1
                    
        new_array.append(final_result)
        final_result=[]
            
    
    return new_array






In [52]:
vector_training_set_question1 =duplicate_entries(training_error1_cf)

TypeError: 'Sequential' object is not subscriptable

In [ ]:
len(vector_training_set_question2)

In [18]:
vector_training_set_question2 =duplicate_entries(training_error2_cf)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:11: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



In [19]:
a = np.array(vector_training_set_question1)
b = np.array(vector_training_set_question2)

In [33]:
a = np.array(training_error1_cf)
b = np.array(training_error2_cf)

In [34]:
result=np.column_stack((a,b))

In [56]:
result[0]


array([list(['step', 'step', 'guid', 'invest', 'share', 'market', 'india']),
       list(['step', 'step', 'guid', 'invest', 'share', 'market'])],
      dtype=object)

In [36]:
in_dim=500

In [38]:
X=result

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.10, random_state=30)

In [25]:
X_train= X_train.reshape(363861, in_dim)

In [26]:
X_train = X_train.astype('float32')

In [27]:
X_test = X_test.reshape(40429, in_dim) 

In [28]:
X_test = X_test.astype('float32')


In [40]:
from keras.models import Sequential
from keras.layers import Dense,Activation
from keras.regularizers import L1L2
from keras.utils import np_utils
from keras.layers import LSTM,Flatten

Using TensorFlow backend.


In [33]:
nb_classes =2

In [47]:
Y_train = np_utils.to_categorical(y_train)
Y_test = np_utils.to_categorical(y_test)

In [45]:
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [48]:
model = Sequential()
#model.add(Dense(2, activation='relu'))
model.add(LSTM(30, dropout_U =0.2,return_sequences=True))
model.add(Flatten())
model.add(Dense(2, activation='softmax') )

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: UserWarning:

Update your `LSTM` call to the Keras 2 API: `LSTM(30, return_sequences=True, recurrent_dropout=0.2)`



In [49]:
model.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['accuracy'])

In [50]:
history = model.fit(X_train, Y_train, validation_split=0.10,epochs=100,batch_size=8)

Train on 327474 samples, validate on 36387 samples
Epoch 1/100


ValueError: setting an array element with a sequence.

In [ ]:
score = model.evaluate(X_test, Y_test,verbose=0)



In [ ]:
score

In [ ]:


print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
print(history.history.keys())

In [ ]:

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
predictions = model.predict(X_test)
print('First prediction:', predictions)

In [ ]:
#KNN

In [53]:
nsamples, nx, ny = X_train.shape
X_train = X_train.reshape((nsamples,nx*ny))

In [54]:
nsamples1, nx1, ny1 = X_test.shape
X_test = X_test.reshape((nsamples1,nx1*ny1))


In [55]:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
scaler.fit(X_train)

X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)

ValueError: setting an array element with a sequence.

In [28]:
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(n_neighbors=2)  
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='uniform')

In [29]:
y_pred = classifier.predict(X_test)

In [30]:
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))

[[25257   136]
 [14970    66]]
             precision    recall  f1-score   support

          0       0.63      0.99      0.77     25393
          1       0.33      0.00      0.01     15036

avg / total       0.52      0.63      0.49     40429



In [35]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

0.6263573177669495


In [ ]:
#Random Forest


In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model= RandomForestRegressor(n_estimators=1000)
# Train the model using the training sets and check score
model.fit(X,y)
#Predict Output

In [ ]:
predicted= model.predict(Y_test)